# Crashes

In [60]:
import pandas as pd
from datetime import datetime

crashes = pd.read_csv('mvc_crashes.csv')
print(crashes.columns.values)

cleaned_crashes = crashes[['COLLISION_ID','CRASH DATE','CRASH TIME','BOROUGH','ZIP CODE','LATITUDE','LONGITUDE','NUMBER OF PERSONS INJURED']].copy()
cleaned_crashes.rename(columns={"COLLISION_ID": "CollisionID",
								"CRASH DATE": "CrashDate",
								"CRASH TIME": "CrashTime",
								"BOROUGH": "Borough",
								"ZIP CODE": "ZipCode",
								"LATITUDE": "Latitude",
								"LONGITUDE": "Longitude",
								"NUMBER OF PERSONS INJURED": "NumberOfInjured"
								}, inplace=True)

C:\Users\Tizian\AppData\Local\Temp\ipykernel_10208\1069871969.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  crashes = pd.read_csv('mvc_crashes.csv')


['CRASH DATE' 'CRASH TIME' 'BOROUGH' 'ZIP CODE' 'LATITUDE' 'LONGITUDE'
 'LOCATION' 'ON STREET NAME' 'CROSS STREET NAME' 'OFF STREET NAME'
 'NUMBER OF PERSONS INJURED' 'NUMBER OF PERSONS KILLED'
 'NUMBER OF PEDESTRIANS INJURED' 'NUMBER OF PEDESTRIANS KILLED'
 'NUMBER OF CYCLIST INJURED' 'NUMBER OF CYCLIST KILLED'
 'NUMBER OF MOTORIST INJURED' 'NUMBER OF MOTORIST KILLED'
 'CONTRIBUTING FACTOR VEHICLE 1' 'CONTRIBUTING FACTOR VEHICLE 2'
 'CONTRIBUTING FACTOR VEHICLE 3' 'CONTRIBUTING FACTOR VEHICLE 4'
 'CONTRIBUTING FACTOR VEHICLE 5' 'COLLISION_ID' 'VEHICLE TYPE CODE 1'
 'VEHICLE TYPE CODE 2' 'VEHICLE TYPE CODE 3' 'VEHICLE TYPE CODE 4'
 'VEHICLE TYPE CODE 5']


In [69]:
crashes_only = cleaned_crashes[['CollisionID','CrashDate','NumberOfInjured']].copy()

# Adds WeatherID to Crashes
#
# crashes_only['CrashDate'] = pd.to_datetime(crashes_only['CrashDate'], format='%m/%d/%Y')
# epoch = datetime(1970, 1, 1)
# for index, row in crashes_only.iterrows():
# 	delta = (row['CrashDate'] - epoch)
# 	crashes_only.loc[index, 'WeatherID'] = delta.days
# crashes_only['WeatherID'] = crashes_only['WeatherID'].astype(int)

crashes_only.drop(columns=['CrashDate'], inplace=True)
print(crashes_only)

crashes_only.to_csv('Crashes.csv', index=False)

         CollisionID  NumberOfInjured
0            4455765              2.0
1            4513547              1.0
2            4541903              0.0
3            4456314              0.0
4            4486609              0.0
...              ...              ...
2024473      4648110              0.0
2024474      4648117              1.0
2024475      4648366              0.0
2024476      4648129              1.0
2024477      4647913              0.0

[2024478 rows x 2 columns]


## Location

Planänderung: Datenbankentwurf wie im Diagramm dargestellt, mit hinzugefügten Fremdschlüsseln in `Neigbourhoods` und `ZipCodes`. Die `ZipCodeID` entfällt, da der `ZipCode` bereits eindeutig ist. Die Fremdschlüssel `BoroughID` und `NeigbourhoodID` entfallen demach in der `Location` Tabelle. Die `ZipCodeID` entfällt/wird direkt durch den `ZipCode` ersetzt. Des weiteren wird die `LocationID` durch die `CollisionID` ersetzt (zwischen `Location` und `Crashes` besteht eine 1:1 Beziehung).

In [ ]:
location = cleaned_crashes[['CollisionID','ZipCode','Latitude','Longitude']].copy()
location[['ZipCode']] = location[['ZipCode']].fillna(0)
location[['ZipCode']] = location[['ZipCode']].replace('', 0)
print(location)
location[['ZipCode']] = location[['ZipCode']].round(0)
location.to_csv('Location.csv', index=False)

## Time

Time hat aktuell ebenfalls `CollisionID` statt `TimeID`.

In [64]:
time = cleaned_crashes[['CollisionID','CrashDate','CrashTime']].copy()
time['CrashDateTime'] = pd.to_datetime(time['CrashDate'] + time['CrashTime'], format='%m/%d/%Y%H:%M')
print("Number of unique days: " + str(time['CrashDate'].nunique()))
print("Earliest date: " + str(time["CrashDateTime"].min()))
print("Latest date: " + str(time["CrashDateTime"].max()))

# Achtung, Berechnung der WeatherID dauert mehrere Minuten (da for-each)
time['CrashDate'] = pd.to_datetime(time['CrashDate'], format='%m/%d/%Y')
epoch = datetime(1970, 1, 1)
for index, row in crashes_only.iterrows():
	delta = (row['CrashDate'] - epoch)
	time.loc[index, 'WeatherID'] = delta.days
time['WeatherID'] = time['WeatherID'].astype(int)


time.drop(columns=['CrashDate', 'CrashTime'], inplace=True)
print(time)

time.to_csv('Time.csv', index=False)

Number of unique days: 4087
Earliest date: 2012-07-01 00:05:00
Latest date: 2023-09-08 23:57:00
         CollisionID       CrashDateTime  WeatherID
0            4455765 2021-09-11 02:39:00      18881
1            4513547 2022-03-26 11:45:00      19077
2            4541903 2022-06-29 06:55:00      19172
3            4456314 2021-09-11 09:35:00      18881
4            4486609 2021-12-14 08:13:00      18975
...              ...                 ...        ...
2024473      4648110 2023-07-03 18:05:00      19541
2024474      4648117 2023-07-22 21:39:00      19560
2024475      4648366 2023-07-02 17:55:00      19540
2024476      4648129 2023-07-22 13:15:00      19560
2024477      4647913 2023-07-22 03:00:00      19560

[2024478 rows x 3 columns]


## Weather

Es gab zwischen dem 01.07.2012 und dem 08.09.2023 an 4087 Tagen Unfälle (sind wahrscheinlich alle Tage in dem Zeitraum). Wetterinformationen sind aktuell einmal pro Tag vorhanden (nicht stündlich).

`WeatherID` wird berechnet über die Anzahl der Tage seit dem 01.01.1970.

Inhalt der CSV:
- mittlere Tagestemperatur in °C
- Sunrise/Sunset nach ISO 8601
- Summe der Regenmenge in mm
- Summe der Schneehöhe in cm
- maximale Windgeschwindigkeit in km/h


In [59]:
from datetime import datetime

weather = pd.read_csv("open-meteo-edit.csv")
weather['time'] = pd.to_datetime(weather['time'])
epoch = datetime(1970, 1, 1)
for index, row in weather.iterrows():
	delta = (row['time'] - epoch)
	weather.loc[index, 'WeatherID'] = delta.days
weather['WeatherID'] = weather['WeatherID'].astype(int)

weather.drop(columns=['time'], inplace=True)
weather.rename(columns={"temperature_mean": "Temperature",
						"sunrise": "Sunrise",
						"sunset": "Sunset",
						"rain_sum": "Rainfall",
						"snowfall_sum": "Snowfall",
						"windspeed_max": "Windspeed"
						}, inplace=True)
print(weather)
weather.to_csv("Weather.csv", index=False)

      Temperature           Sunrise            Sunset  Rainfall  Snowfall  \
0            27.4  2012-07-01T05:28  2012-07-01T20:30       1.7       0.0   
1            25.7  2012-07-02T05:29  2012-07-02T20:30       0.0       0.0   
2            25.8  2012-07-03T05:30  2012-07-03T20:30       0.0       0.0   
3            26.8  2012-07-04T05:30  2012-07-04T20:30       3.9       0.0   
4            28.9  2012-07-05T05:31  2012-07-05T20:30       0.0       0.0   
...           ...               ...               ...       ...       ...   
4084         27.9  2023-09-06T06:27  2023-09-06T19:20       0.0       0.0   
4085         27.6  2023-09-07T06:28  2023-09-07T19:19       0.5       0.0   
4086         26.0  2023-09-08T06:29  2023-09-08T19:17       7.9       0.0   
4087         23.9  2023-09-09T06:30  2023-09-09T19:15      13.7       0.0   
4088         22.5  2023-09-10T06:31  2023-09-10T19:14       8.8       0.0   

      Windspeed  WeatherID  
0          17.0      15522  
1          14.0  

# Vehicles

In [1]:
import pandas as pd
vehicles = pd.read_csv('mvc_vehicles.csv')

C:\Users\Tizian\AppData\Local\Temp\ipykernel_10208\3228589016.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles = pd.read_csv('mvc_vehicles.csv')


         UNIQUE_ID  COLLISION_ID  CRASH_DATE CRASH_TIME  \
0         10385780        100201  09/07/2012       9:03   
1         19140702       4213082  09/23/2019       8:15   
2         14887647       3307608  10/02/2015      17:18   
3         14889754       3308693  10/04/2015      20:34   
4         14400270        297666  04/25/2013      21:15   
...            ...           ...         ...        ...   
3704401   20083223       4466593  10/11/2021      11:00   
3704402   20083044       4466424  10/11/2021       7:15   
3704403   20081526       4465968  10/11/2021       0:15   
3704404   20083216       4466592  10/11/2021      22:10   
3704405   20083752       4466794  10/08/2021      23:30   

                                   VEHICLE_ID STATE_REGISTRATION  \
0                                           1                 NY   
1        0553ab4d-9500-4cba-8d98-f4d7f89d5856                 NY   
2                                           2                 NY   
3                  

In [2]:
cleaned_vehicles = vehicles[['UNIQUE_ID','COLLISION_ID','VEHICLE_ID','VEHICLE_TYPE','VEHICLE_YEAR','VEHICLE_MAKE','VEHICLE_MODEL']].copy()
cleaned_vehicles.rename(columns={"UNIQUE_ID": "UniqueID",
								"COLLISION_ID": "CollisionID",
								"VEHICLE_ID": "VehicleID",
								"VEHICLE_TYPE": "VehicleType",
								"VEHICLE_YEAR": "VehicleYear",
								"VEHICLE_MAKE": "VehicleMake",
								"VEHICLE_MODEL": "VehicleModel"
								}, inplace=True)
cleaned_vehicles.to_csv('cleaned_vehicles.csv', index=False)

# Persons

In [6]:
import pandas as pd
persons = pd.read_csv('mvc_persons.csv')

In [7]:
cleaned_persons = persons[['UNIQUE_ID', 'PERSON_ID','COLLISION_ID','VEHICLE_ID','PERSON_AGE','PERSON_INJURY','EMOTIONAL_STATUS','PERSON_SEX','PED_ROLE']].copy()
cleaned_persons.rename(columns={"UNIQUE_ID": "UniqueID",
								"PERSON_ID": "PersonID",
								"COLLISION_ID": "CollisionID",
								"VEHICLE_ID": "VehicleID",
								"PERSON_AGE": "Age",
								"PERSON_INJURY": "Injury",
								"EMOTIONAL_STATUS": "EmotionalStatus",
								"PERSON_SEX": "Gender",
								"PED_ROLE": "PedRole"
								}, inplace=True)
cleaned_persons.to_csv('cleaned_person.csv', index=False)